Loading the Dataset

In [1]:
import pandas as pd
import numpy as np

# 1. Load the raw data (fast parse)
print("Loading dataset...")
df_raw = pd.read_csv('../data/raw/time_series_60min_singleindex.csv', 
                     parse_dates=['utc_timestamp'], 
                     index_col='utc_timestamp')

Loading dataset...


Dropping Unnecessary Columns

In [8]:
contract_columns = {
    # Germany (Main Analysis)
    'DE_load_actual_entsoe_transparency':    'Load_DE',
    'DE_solar_generation_actual':            'Solar_DE',
    'DE_wind_generation_actual':             'Wind_DE',
    'DE_LU_price_day_ahead':                 'Price_DE', 
    
    # France (Comparison A)
    'FR_load_actual_entsoe_transparency':    'Load_FR',
    'FR_solar_generation_actual':            'Solar_FR',
    
    # Great Britain (Comparison B - Island Grid)
    'GB_GBN_load_actual_entsoe_transparency':'Load_UK'
}

# 3. Create the reduced dataframe
df = df_raw[contract_columns.keys()].rename(columns=contract_columns)

# 4. Filter for the "Goldilocks" Era (2018-2019)
# This removes the spotty early data and the COVID era
df = df['2019-01-01':'2019-12-31']

print("Reduction Complete.")
print(f"Old Shape: {df_raw.shape}")
print(f"New Shape: {df.shape}")
df.head()

Reduction Complete.
Old Shape: (50401, 299)
New Shape: (8760, 7)


,Load_DE,Solar_DE,Wind_DE,Price_DE,Load_FR,Solar_FR,Load_UK
utc_timestamp,,,,,,,
2019-01-01 00:00:00+00:00,41562.0,0.0,24905.0,10.07,60301.0,0.0,27090.0
2019-01-01 01:00:00+00:00,40100.0,0.0,25208.0,-4.08,58540.0,0.0,26958.0
2019-01-01 02:00:00+00:00,38883.0,0.0,26568.0,-9.91,55144.0,0.0,24546.0
2019-01-01 03:00:00+00:00,38806.0,0.0,28500.0,-7.41,52978.0,0.0,22100.0
2019-01-01 04:00:00+00:00,38593.0,0.0,29808.0,-12.55,52584.0,0.0,21038.0


Missing Values Check

In [9]:
# Check for nulls in our specific time window
print("--- MISSING VALUES (BEFORE CLEANING) ---")
null_report = df.isnull().sum()
print(null_report)

# Calculate total missing %
total_cells = df.size
total_missing = null_report.sum()
print(f"\nTotal Missing Cells: {total_missing}")
print(f"Dataset Completeness: {100 - (total_missing/total_cells)*100:.4f}%")

--- MISSING VALUES (BEFORE CLEANING) ---
Load_DE     0
Solar_DE    0
Wind_DE     0
Price_DE    2
Load_FR     5
Solar_FR    5
Load_UK     0
dtype: int64

Total Missing Cells: 12
Dataset Completeness: 99.9804%


Handling Missing Values

In [10]:
# 1. IMPUTATION
# Logic: Electricity usage is continuous. If 2pm is missing, 1pm is the best guess.
df_clean = df.fillna(method='ffill')

# 2. FEATURE ENGINEERING (Derivable Attributes)
# Total Renewables (Wind + Solar)
df_clean['Renewables_DE'] = df_clean['Solar_DE'] + df_clean['Wind_DE']

# Net Load (The "Stress" on the fossil fuel grid)
df_clean['Net_Load_DE'] = df_clean['Load_DE'] - df_clean['Renewables_DE']

# 3. FINAL VERIFICATION
print("--- MISSING VALUES (AFTER CLEANING) ---")
print(df_clean.isnull().sum())

if df_clean.isnull().sum().sum() == 0:
    print("\n DATASET IS CLEAN AND READY.")
else:
    print("\n WARNING: Some nulls remain (likely at the very start of the file).")
    # If any remain at index 0, backfill them once
    df_clean = df_clean.fillna(method='bfill')

--- MISSING VALUES (AFTER CLEANING) ---
Load_DE          0
Solar_DE         0
Wind_DE          0
Price_DE         0
Load_FR          0
Solar_FR         0
Load_UK          0
Renewables_DE    0
Net_Load_DE      0
dtype: int64

 DATASET IS CLEAN AND READY.


C:\Users\Rayyan Tech\AppData\Local\Temp\ipykernel_11852\2303185634.py:3: FutureWarning: DataFrame.fillna with 'method' is deprecated and will raise in a future version. Use obj.ffill() or obj.bfill() instead.
  df_clean = df.fillna(method='ffill')


In [11]:
# Check for nulls again
print("--- MISSING VALUES (BEFORE CLEANING) ---")
null_report = df_clean.isnull().sum()
print(null_report)

# Calculate total missing %
total_cells = df.size
total_missing = null_report.sum()
print(f"\nTotal Missing Cells: {total_missing}")
print(f"Dataset Completeness: {100 - (total_missing/total_cells)*100:.4f}%")

--- MISSING VALUES (BEFORE CLEANING) ---
Load_DE          0
Solar_DE         0
Wind_DE          0
Price_DE         0
Load_FR          0
Solar_FR         0
Load_UK          0
Renewables_DE    0
Net_Load_DE      0
dtype: int64

Total Missing Cells: 0
Dataset Completeness: 100.0000%


Saving The Cleaned Dataset

In [12]:
# Save to processed folder
output_path = '../data/processed/electricity_cleaned.csv'
df_clean.to_csv(output_path)
print(f"File saved to: {output_path}")

File saved to: ../data/processed/electricity_cleaned.csv
